<a href="https://colab.research.google.com/github/edwinmgallego/Best-README-Template/blob/main/avd_121_Mineria_de_datos_clase03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>carga inicial de la librerias</b></h1>
→

In [2]:
# 📌 1. Importar librerías necesarias
!pip install surprise  # Instalamos la librería si no está instalada

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505160 sha256=38ec4838099252022c1be108d953aa498e98bca746407db32adcddc0fefdf17d
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


Importacion de librerias

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

In [7]:
# 📌 2. Cargar el dataset de Netflix
df = pd.read_csv("netflix_titles.csv")
print(df)
df = df.dropna(subset=["cast", "rating", "director"])  # Eliminamos valores nulos
df["date_added"] = pd.to_datetime(df["date_added"].str.strip(), format='mixed')
  # Convertimos fechas
df["popularity"] = df.groupby("title")["show_id"].transform("count")
df["popularity"] = (df["popularity"] - df["popularity"].min()) / (df["popularity"].max() - df["popularity"].min())

# 📌 3. Preparar los datos para Surprise
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df[["show_id", "title", "popularity"]], reader)
trainset = data.build_full_trainset()

# 📌 4. Crear y entrenar el modelo SVD
model = SVD()
model.fit(trainset)

# 📌 5. Evaluación del modelo
cross_validate(model, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

# 📌 6. Función para recomendar películas
def recomendar_peliculas(usuario_id, df, modelo, n_recomendaciones=5):
    peliculas_no_vistas = df[["show_id", "title"]].drop_duplicates()
    peliculas_no_vistas["predicted_rating"] = peliculas_no_vistas["show_id"].apply(lambda x: modelo.predict(usuario_id, x).est)
    recomendaciones = peliculas_no_vistas.sort_values("predicted_rating", ascending=False).head(n_recomendaciones)
    return recomendaciones

# 📌 7. Generar recomendaciones para un usuario ficticio (ID = 1001)
recomendaciones = recomendar_peliculas(1001, df, model)
print("🎬 Recomendaciones para el usuario 1001:")
print(recomendaciones)


     show_id     type                  title         director  \
0         s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1         s2  TV Show          Blood & Water              NaN   
2         s3  TV Show              Ganglands  Julien Leclercq   
3         s4  TV Show  Jailbirds New Orleans              NaN   
4         s5  TV Show           Kota Factory              NaN   
...      ...      ...                    ...              ...   
8802   s8803    Movie                 Zodiac    David Fincher   
8803   s8804  TV Show            Zombie Dumb              NaN   
8804   s8805    Movie             Zombieland  Ruben Fleischer   
8805   s8806    Movie                   Zoom     Peter Hewitt   
8806   s8807    Movie                 Zubaan      Mozez Singh   

                                                   cast        country  \
0                                                   NaN  United States   
1     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa 